In [1]:
import sqlite3
from data_algebra.data_ops import *  # https://github.com/WinVector/data_algebra
import data_algebra.SQLite


conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# From: 
#   https://docs.python.org/3.5/library/sqlite3.html

cur.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

In [2]:
# work a simple example

ops = TableDescription('stocks', 
                 ['date',
                  'trans',
                  'symbol',
                  'qty',
                  'price']) .\
    extend({'cost': 'qty * price'})

print(ops.to_python(pretty=True))

TableDescription(
    table_name="stocks", column_names=["date", "trans", "symbol", "qty", "price"]
).extend({"cost": "qty * price"})



In [3]:
db_model = data_algebra.SQLite.SQLiteModel()

sql = ops.to_sql(db_model, pretty=True)
print(sql)

SELECT "date",
       "trans",
       "price",
       "symbol",
       "qty",
       "qty" * "price" AS "cost"
FROM
  (SELECT "symbol",
          "date",
          "trans",
          "price",
          "qty"
   FROM "stocks") "SQ_0"


In [4]:
db_model.read_query(conn, sql)

,date,trans,price,symbol,qty,cost
0,2006-01-05,BUY,35.14,RHAT,100.0,3514.0


In [5]:
# neaten up
conn.close()